In [1]:
class Simple_node:
    
    def __init__(self, node_id):
        self.node_id = node_id
        self.list_of_neighbours = []
        self.neighbours = self.list_of_neighbours
        
    def add_neighbour(self, neighbouring_node_id): 
        tf = neighbouring_node_id in self.list_of_neighbours
        if tf == False:
            self.list_of_neighbours.append(neighbouring_node_id)
        return self.list_of_neighbours

In [2]:
class Node_and_weights:
    
    def __init__(self, node_id):
        self.node_id = node_id
        self.list_of_neighbours = []
        self.neighbours = self.list_of_neighbours
        
    def add_neighbour(self, neighbouring_node_id_and_weight): 
        tf = False
        for el in self.list_of_neighbours:
            if el[0] == neighbouring_node_id_and_weight[0]:
                tf = True
        if tf == False:
            self.list_of_neighbours.append(neighbouring_node_id_and_weight)
        return self.list_of_neighbours

In [3]:
map_of_nodes = {}

def graph_loading(file_name, class_name, dictionary = map_of_nodes):
    file = open(file_name)
    content = file.read()
    file.close()
    lines = content.split('\n')
    
    list_of_lists = []
    for el in lines:
        list_of_lists.append(el.split(' '))
        
    finished_list_of_connected_nodes = []
    for el in list_of_lists:
        flocn = []
        for e in el:
            flocn.append(int(e))
        finished_list_of_connected_nodes.append(flocn)
        
    for el in finished_list_of_connected_nodes:
        dictionary[el[0]] = class_name(el[0])
        dictionary[el[1]] = class_name(el[1])
        
    keys = dictionary.keys()
    
    for el in finished_list_of_connected_nodes:
        for key in keys:
            if len(el) == 2:
                if key == el[0]:
                    dictionary[el[0]].add_neighbour(el[1])
                elif key == el[1]:
                    dictionary[el[1]].add_neighbour(el[0])
            else:
                if key == el[0]:
                    dictionary[el[0]].add_neighbour([el[1], el[2]])
                elif key == el[1]:
                    dictionary[el[1]].add_neighbour([el[0], el[2]])
                    
    return dictionary

### the shortest path

In [4]:
def shortest_path(start, graph): # the place we want our shortest path to start, graph made with function 'graph_loading'
    
    nodes = list(graph.keys()) # all nodes
    
    # creating a dictionary with edges and their weight
    # node: [neighbour, weight of the edge]
    graph_edges = []
    for el in nodes:
        graph_edges.append([el, graph[el].neighbours])
    graph_edges_dict = {}
    for el in graph_edges:
        graph_edges_dict[el[0]] = el[1]
        
    shortest_path = {} # shortest path to specific node from the starting node
    where_we_came_from = {} # which node we came from
    for el in nodes:
        shortest_path[el] = -1 # for now (weights - from 0)
        where_we_came_from[el] = 0 # for now (nodes - form 1)
        
    changes = {} # dictionary with changes
    if sum(shortest_path.values()) == len(nodes)*(-1): # if it's true then it's the begginning
        step = 0 # starting step
        shortest_path[start] = 0 # we're changing the value of the shortest path from the starting node to the same node to 0
        changed = [] # list of changes
        
        for neighbours in graph_edges_dict[start]: # for all neighbours of the starting node
            # now it's the starting strp so we know that all the shortest paths are -1 (except the one we changed to 0)
            # so there's no need to check if the previous path was shorter (the previous one doesn't exist)
            shortest_path[neighbours[0]] = neighbours[1] # we're changing the shortest paths to the weight of edges
            where_we_came_from[neighbours[0]] = start # we came from the starting node
            changed.append(neighbours[0]) # saving the info of the changed nodes
        changes[step] = changed # saving the info of the changes that took place in this specific step
        step = 1 # changing the step
        
    if step > 0:
        while len(changes[step-1]) != 0: # stopping when nothing changes anymore
            changes_2 = [] 
            
            for node in changes[step-1]: # if the node was changed in the previous step
                for neighbours in graph_edges_dict[node]: # for all neighbours of the selected node
                    m = shortest_path[node] + neighbours[1] # shortest path through this selected node
                    # and if the path to the specific neighbour is -1
                    # or the path that we have is greater than m
                    if shortest_path[neighbours[0]] == -1 or shortest_path[neighbours[0]] > m:
                        shortest_path[neighbours[0]] = m # then we're changing the shortest path to m
                        where_we_came_from[neighbours[0]] = node # and the node that we came from to the one we selected earlier
                        changes_2.append(neighbours[0]) # saving the info of the changed nodes
                
                changes[step] = changes_2 # saving the info of the changes that took place in this specific step
                step = step + 1 # going to the next step
                
    return (shortest_path, where_we_came_from) # returning the shortest path and the nodes that we came from

In [5]:
map_1 = {}
loaded_graph = graph_loading('graph_1.txt', Node_and_weights, map_1)
loaded_graph

{1: <__main__.Node_and_weights at 0x17099ad9940>,
 2: <__main__.Node_and_weights at 0x17099ad99d0>,
 4: <__main__.Node_and_weights at 0x17099ad9af0>,
 3: <__main__.Node_and_weights at 0x17099ad97f0>,
 5: <__main__.Node_and_weights at 0x17099ad9070>,
 6: <__main__.Node_and_weights at 0x17099ad9220>}

In [6]:
shortest_path(1, loaded_graph)

({1: 0, 2: 1, 4: 9, 3: 7, 5: 4, 6: 8}, {1: 0, 2: 1, 4: 5, 3: 1, 5: 2, 6: 3})

### components

In [7]:
def dict_from_list(list_1):
    components_dict = {}
    for el in list_1:
        components_dict[el[0]] = el[1]
    return components_dict

In [8]:
def does_it_change(list_1):
    is_it_the_same = []
    for el in list_1:
        is_it_the_same.append(el[1][0] == el[1][1])
    return sum(is_it_the_same) != len(list_1)

In [9]:
def components(graph):
    components = []
    
    # checking the neighbours - beginning
    for el in graph:
        components.append([el, [999, min(min(graph[el].neighbours), el)]])
        
    components_dict = dict_from_list(components)
    
    # distant phase - beginning
    for coordinates in components:
        if coordinates[1][1] >= components_dict[coordinates[1][1]][1]:
            coordinates[1][0] = components_dict[coordinates[1][1]][1]
     
    # deleting 999
    for a in components:
        if a[1][0] == 999:
            a[1][0] = a[1][1]
            
    # stopping when nothing changes anymore
    while does_it_change(components) == True:
        components_dict = dict_from_list(components)
        
        # neighbours phase
        for coordinate in components:
            for neighbour in graph[coordinate[0]].neighbours:
                if coordinate[1][0] >= components_dict[neighbour][0]:
                    coordinate[1][1] = coordinate[1][0]
                    coordinate[1][0] = components_dict[neighbour][0]
        components_dict = dict_from_list(components)
        # distant phase
        for coordinates in components:
            if coordinates[1][1] >= components_dict[coordinates[1][1]][1]:
                coordinates[1][1] = coordinates[1][0]
                coordinates[1][0] = components_dict[coordinates[1][1]][1]
        
    return(components)

In [10]:
map_2 = {}
loaded_graph_2 = graph_loading('graph_2.txt', Simple_node, map_2)
loaded_graph_2

{1: <__main__.Simple_node at 0x17099ad3730>,
 2: <__main__.Simple_node at 0x17099ad3ca0>,
 3: <__main__.Simple_node at 0x17099ad3cd0>,
 4: <__main__.Simple_node at 0x17099ad3250>,
 9: <__main__.Simple_node at 0x17099ad31c0>,
 5: <__main__.Simple_node at 0x17099ad3af0>,
 8: <__main__.Simple_node at 0x17099ad3280>,
 6: <__main__.Simple_node at 0x17099ad3610>,
 7: <__main__.Simple_node at 0x17099ad3760>,
 10: <__main__.Simple_node at 0x17099ad3d30>}

In [11]:
components(loaded_graph_2)

[[1, [1, 1]],
 [2, [1, 1]],
 [3, [3, 3]],
 [4, [3, 3]],
 [9, [3, 3]],
 [5, [3, 3]],
 [8, [3, 3]],
 [6, [6, 6]],
 [7, [6, 6]],
 [10, [6, 6]]]

### ----------------------------------------------------------------------------------------------------------------------------------

In [12]:
from random import randint

In [13]:
graph_3 = []

for i in range(2000):
    graph_3.append(randint(1,1000))

In [14]:
filepath = 'graph_3.txt'
f = open(filepath, 'w')
i = 0
while(i<len(graph_3)):
    f.write(str(graph_3[i]))
    f.write(' ')
    f.write(str(graph_3[i+1]))
    i = i+2
    if i!=len(graph_3):
        f.write('\n')
f.close()

In [15]:
def components_2(graph):
    components = []
    
    # checking the neighbours - beginning
    for el in graph:
        components.append([el, [999, min(min(graph[el].neighbours), el)]])
        
    components_dict = dict_from_list(components)
    
    # distant phase - beginning
    for coordinates in components:
        if coordinates[1][1] >= components_dict[coordinates[1][1]][1]:
            coordinates[1][0] = components_dict[coordinates[1][1]][1]
     
    # deleting 999
    for a in components:
        if a[1][0] == 999:
            a[1][0] = a[1][1]
            
    loop = 1
            
    # stopping when nothing changes anymore
    while does_it_change(components) == True:
        components_dict = dict_from_list(components)
        
        # neighbours phase
        for coordinate in components:
            for neighbour in graph[coordinate[0]].neighbours:
                if coordinate[1][0] >= components_dict[neighbour][0]:
                    coordinate[1][1] = coordinate[1][0]
                    coordinate[1][0] = components_dict[neighbour][0]
        components_dict = dict_from_list(components)
        # distant phase
        for coordinates in components:
            if coordinates[1][1] >= components_dict[coordinates[1][1]][1]:
                coordinates[1][1] = coordinates[1][0]
                coordinates[1][0] = components_dict[coordinates[1][1]][1]
                
        loop = loop + 1
        
    return([loop, components])

In [16]:
map_3 = {}
loaded_graph_3 = graph_loading('graph_3.txt', Simple_node, map_3)

In [17]:
components_2(loaded_graph_3)

[7,
 [[197, [1, 1]],
  [614, [1, 1]],
  [863, [1, 1]],
  [754, [1, 1]],
  [523, [1, 1]],
  [633, [1, 1]],
  [190, [1, 1]],
  [886, [1, 1]],
  [117, [1, 1]],
  [473, [1, 1]],
  [745, [1, 1]],
  [803, [1, 1]],
  [330, [1, 1]],
  [691, [1, 1]],
  [836, [1, 1]],
  [937, [1, 1]],
  [692, [1, 1]],
  [718, [1, 1]],
  [345, [1, 1]],
  [434, [1, 1]],
  [49, [1, 1]],
  [788, [1, 1]],
  [619, [19, 19]],
  [19, [19, 19]],
  [478, [1, 1]],
  [970, [1, 1]],
  [188, [1, 1]],
  [579, [1, 1]],
  [195, [1, 1]],
  [221, [1, 1]],
  [890, [1, 1]],
  [300, [1, 1]],
  [224, [224, 224]],
  [373, [224, 224]],
  [664, [1, 1]],
  [913, [1, 1]],
  [226, [1, 1]],
  [489, [1, 1]],
  [552, [1, 1]],
  [202, [1, 1]],
  [582, [1, 1]],
  [904, [1, 1]],
  [77, [1, 1]],
  [813, [1, 1]],
  [389, [1, 1]],
  [180, [1, 1]],
  [539, [1, 1]],
  [509, [1, 1]],
  [987, [1, 1]],
  [964, [1, 1]],
  [426, [1, 1]],
  [879, [1, 1]],
  [683, [1, 1]],
  [306, [1, 1]],
  [377, [1, 1]],
  [220, [1, 1]],
  [977, [1, 1]],
  [739, [1, 1]],
 